## Multi-objective optimization of a dogbone design
---
 Based on "ASTM D638-14: Standard Test Method for Tensile Properties of Plastics 1"
 ASTM D638 Type I test specimen (165x19mm)


In [23]:
# definining the dogbone mesh with gmsh: only outline, no perimeters

import gmsh
import pathlib
from pathlib import Path

run_GUI = 1  # open graphical user interface?

LO = 165  # overall length
WO = 19   # width overall
WN = 13   # narrow section width
LN = 57   # length of the narrow section
RF = 76   # radius of the fillet
G = 50    # gage length (streight length)
T = 3.2   # thickness
A = 36.36 # approximated gauge end

GL = (LO - 50)/2  # gage step (from grip to narrow)
DF = (WO-WN)/2    # side difference between narrow and grip

num_el_x, num_el_y = 20, 10  # adjust the num of elements
dx, dy = LO / num_el_x, WO / num_el_y

# initialize Gmsh
if not gmsh.is_initialized():
    gmsh.initialize()

# path setup
meshname = "dogbone_type1"
gmsh.model.add(meshname)
cwd = Path.cwd()
meshpath = cwd / "MOEA_meshes" / "dogbones" / f"{meshname}.msh"
meshpath_stl = cwd / "MOEA_meshes" / "dogbones" / f"{meshname}.stl"

# defining a namespace
gmo = gmsh.model.occ  # gmo ~ gmsh.model.opencascade
gmg = gmsh.model.geo  # gmg ~ gmsh.model.geo

# mesh sizes
lc1 = dx    # mesh size grips
lc2 = dx/2  # mesh size center section

# defining the points
gmo.addPoint(0, 0, 0, lc1, 1)
gmo.addPoint(0, LO, 0, lc1, 2)
gmo.addPoint(WO, LO, 0, lc1, 3)
gmo.addPoint(WO, 0, 0, lc1, 4)
gmo.addPoint(0, A, 0, lc1, 5)
gmo.addPoint(0, LO-A, 0, lc1, 6)
gmo.addPoint(WO, LO-A, 0, lc1, 7)
gmo.addPoint(WO, A, 0, lc1, 8)
gmo.addPoint(DF, GL, 0, lc2, 9)
gmo.addPoint(DF, LO-GL, 0, lc2, 10)
gmo.addPoint(WO-DF, LO-GL, 0, lc2, 11)
gmo.addPoint(WO-DF, GL, 0, lc2, 12)
gmo.addPoint(DF-RF, GL, 0, lc1, 13)
gmo.addPoint(DF-RF,LO-GL, 0, lc1, 14)
gmo.addPoint(WO-DF+RF, LO-GL, 0, lc1, 15)
gmo.addPoint(WO-DF+RF, GL, 0, lc1, 16)

# adding lines
gmo.addLine(1, 5, 1)
gmo.addLine(9, 10, 2)
gmo.addLine(6, 2, 3)
gmo.addLine(2, 3, 4)
gmo.addLine(3, 7, 5)
gmo.addLine(11, 12, 6)
gmo.addLine(8, 4, 7)
gmo.addLine(4, 1, 8)

gmo.addEllipseArc(9, 13, 9, 5, 9)
gmo.addEllipseArc(10, 14, 10, 6, 10)
gmo.addEllipseArc(11, 15, 11, 7, 11)
gmo.addEllipseArc(12, 16, 12, 8, 12)

# defining a new surface:
gmo.addCurveLoop([1, -9, 2, 10, 3, 4, 5, -11, 6, 12, 7, 8], 20)
gmo.addPlaneSurface([20], 21)  # full surface
gmo.synchronize()

# Extruding surface
vol = gmo.extrude([(2,21)], 0, 0, T)








# global meshing options
gmsh.option.setNumber("Mesh.Algorithm", 11)  # Frontal-Delaunay for 2D meshes

# To use quadrilateral mesh elements:
#gmg.mesh.setRecombine(1, 1)


# apply elliptic smoother to the mesh grid:
#gmsh.option.setNumber("Mesh.Smoothing", 1)

# generate the mesh
gmo.synchronize()
gmsh.model.mesh.generate(3)

# saving the mesh
gmsh.write(str(meshpath))
gmsh.write(str(meshpath_stl))

# Launch the GUI
if run_GUI:
    gmo.synchronize()
    gmsh.fltk.run()

# finish the GMSH session
gmsh.finalize()

Info    : Build background mesh and guiding field ...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 5 (Line)
Info    : [ 20%] Meshing curve 6 (Line)
Info    : [ 20%] Meshing curve 7 (Line)
Info    : [ 20%] Meshing curve 8 (Line)
Info    : [ 30%] Meshing curve 9 (Ellipse)
Info    : [ 30%] Meshing curve 10 (Ellipse)
Info    : [ 30%] Meshing curve 11 (Ellipse)
Info    : [ 40%] Meshing curve 12 (Ellipse)
Info    : [ 40%] Meshing curve 13 (Line)
Info    : [ 40%] Meshing curve 14 (Line)
Info    : [ 40%] Meshing curve 15 (Line)
Info    : [ 50%] Meshing curve 16 (Line)
Info    : [ 50%] Meshing curve 17 (Ellipse)
Info    : [ 50%] Meshing curve 18 (Line)
Info    : [ 50%] Meshing curve 19 (Line)
Info    : [ 60%] Meshing curve 20 (Line)
Info    : [ 60%] Meshing curve 21 (Ellipse)
Info    : [ 60%] Meshing curve 22 (Line)
Info    : [